## Cartopy を利用した地理情報の可視化

[Cartopy](http://scitools.org.uk/cartopy/) は地図生成や地理空間可視化を目的とした地理空間データ処理パッケージである。

以下の例は、Cartopy を利用して、[Natural Earth](http://www.naturalearthdata.com) から日本の都道府県の境界データを取得し、
matplotlib の patch を作成、描画するプログラムである:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shapereader

# 10m 解像度のデータ読み込み
shpfilename = shapereader.natural_earth(resolution='10m',
                                        category='cultural',
                                        name='admin_1_states_provinces')

# 国内の都道府県のみ取得
reader = shapereader.Reader(shpfilename)
provinces = []
for province in reader.records():
    if(province.attributes["admin"] == "Japan"):
        provinces.append(province)

# Figure, Subplot 生成
plt.figure(figsize=[8,8])
ax = plt.axes(projection=ccrs.PlateCarree())

# 都道府県のパッチで埋めていく
for province in provinces:
    geometry = province.geometry
    ax.add_geometries(geometry, ccrs.PlateCarree(), edgecolor="black", linestyle=":")

# 日本以外を含む海岸線を描画
# ax.coastlines(resolution='10m')

# 描画範囲を緯度、経度で指定
ax.set_extent([135, 142, 33, 40])
plt.show()

都道府県データから、英語名 "name"、日本語名 "name_local" を確認してみる。残念ながら　"Shizuoka" に対応する日本語名はないようだ:

In [ ]:
for province in provinces:
    print(province.attributes["name"], province.attributes["name_local"])

### 階級区分図(Choropleth map)による可視化

[住民基本台帳に基づく人口動態データ](http://www.soumu.go.jp/menu_news/s-news/01gyosei02_02000148.html)の人口増減率から都道府県単位の階級区分図（Choropleth_map）を描いてみる:

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shapereader
import pandas as pd
import numpy as np

# とにかく Unnamed 列の文字列を消去するための関数
def rename_unnamed(df): 
    """Rename unamed columns name for Pandas DataFrame

    See https://stackoverflow.com/questions/41221079/rename-multiindex-columns-in-pandas

    Parameters
    ----------
    df : pd.DataFrame object
        Input dataframe

    Returns
    -------
    pd.DataFrame
        Output dataframe

    """
    for i, columns in enumerate(df.columns.levels):
        columns_new = columns.tolist()
        for j, row in enumerate(columns_new):
            if "Unnamed: " in row:
                columns_new[j] = ""
        if pd.__version__ < "0.21.0":  # https://stackoverflow.com/a/48186976/716469
            df.columns.set_levels(columns_new, level=i, inplace=True)
        else:
            df = df.rename(columns=dict(zip(columns.tolist(), columns_new)),
                           level=i)
    return df


#　Excel データ読み込み
df = pd.read_excel("000494956.xls", sheet_name=0, header=[1,2,3], skiprows=[4])
# Omit "Unnamed" indices and adjust 
df = rename_unnamed(df)
df.set_index("都道府県名", inplace=True)
df.index.name="都道府県名"

shpfilename = shapereader.natural_earth(resolution='10m',
                                        category='cultural',
                                        name='admin_1_states_provinces')

reader = shapereader.Reader(shpfilename)

provinces = []
for province in reader.records():
    if(province.attributes["admin"] == "Japan"):
        provinces.append(province)

plt.figure(figsize=[8,8])
ax = plt.axes(projection=ccrs.PlateCarree())

#　カラーマップのためのスケールデータ
cmap = plt.cm.magma
cnorm = max(df["平成28年","増減率"]) - min(df["平成28年","増減率"])
coff = min(df["平成28年","増減率"])

for province in provinces:
    geometry = province.geometry
    if province.attributes["name_local"] == "":
        continue
    color = cmap((df["平成28年","増減率"][province.attributes["name_local"]] - coff) / cnorm)
    ax.add_geometries(geometry, ccrs.PlateCarree(), edgecolor="black", linestyle=":",
                        facecolor=color)

ax.set_title("Population Growth Rate H28 - H29")

#ax.coastlines(resolution='10m')
ax.set_extent([125, 146, 25, 47])

sm = plt.cm.ScalarMappable(cmap=cmap,norm=plt.Normalize(cnorm + coff,coff))
sm._A = []
plt.colorbar(sm,ax=ax)

plt.savefig("sample.pdf", dps = 900, bbox_inches="tight")
plt.show()